In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
from catboost import CatBoostRegressor
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import time

In [15]:
test = pd.read_csv(ROOT+ "data/test-proc-v2.csv")

In [3]:
ROOT = "/root/air/"
train = pd.read_csv(ROOT+ "data/train-proc-v2.csv")
test = pd.read_csv(ROOT+ "data/test-proc-v2.csv")

In [4]:
test_sub = pd.read_csv(ROOT+ "sub/grid1/23.0429_catboost-gridf_2020-05-22_15:48:00.csv")

In [5]:
test.target = test_sub.target

In [6]:
stack_all = pd.concat([train,test], axis=0)

In [7]:
stack_all.shape

(20574, 771)

In [8]:
cat_vars = ['min_atmos_press', 'median_atmos_press', 'mean_atmos_press', 'median_wind_spd', 'newtemp64', 'atmospherepressure33', 'min_temp', 'atmospherepressure56', 'median_wind_dir', 'atmospherepressure92', 'max_wind_spd', 'min_wind_spd', 'mean_wind_dir', 'max_rel_humidity', 'newtemp16', 'mean_precip', 'atmospherepressure105', 'max_wind_dir', 'ptp_wind_spd', 'var_wind_dir', 'median_temp', 'newtemp39', 'ptp_atmos_press', 'newtemp0', 'max_temp', 'newtemp23', 'windspeed119', 'location', 'newtemp92', 'atmospherepressure34', 'newtemp57', 'newtemp2', 'atmospherepressure104', 'min_wind_dir', 'atmospherepressure32', 'atmospherepressure20', 'std_wind_dir', 'windspeed95', 'newtemp103', 'ptp_precip', 'atmospherepressure8', 'var_rel_humidity', 'newtemp73', 'newtemp104', 'newtemp49', 'newtemp25', 'atmospherepressure86', 'min_rel_humidity', 'ptp_rel_humidity', 'newrel_humidity34', 'atmospherepressure2', 'newtemp29', 'ptp_wind_dir', 'windspeed47', 'newrel_humidity32', 'windspeed71', 'newtemp3', 'newwind_dir103', 'atmospherepressure12', 'max_precip', 'var_temp', 'max_atmos_press', 'atmospherepressure44', 'std_atmos_press', 'newtemp81', 'newtemp115', 'std_precip', 'std_rel_humidity', 'windspeed120', 'atmospherepressure116', 'newtemp120', 'newrel_humidity2', 'newrel_humidity0', 'newrel_humidity114', 'newtemp58', 'newtemp66', 'newwind_dir56', 'atmospherepressure87', 'newrel_humidity11', 'ptp_temp', 'newrel_humidity78', 'atmospherepressure98', 'newrel_humidity8', 'median_rel_humidity', 'mean_temp', 'newtemp74', 'atmospherepressure9', 'newtemp54', 'newtemp47', 'newtemp52', 'newrel_humidity59', 'newtemp50', 'atmospherepressure7', 'std_temp', 'newwind_dir90', 'newwind_dir54', 'atmospherepressure3', 'mean_wind_spd', 'newrel_humidity10', 'mean_rel_humidity', 'newtemp101', 'newwind_dir3', 'newrel_humidity1', 'newtemp41', 'newwind_dir120', 'newrel_humidity95', 'atmospherepressure85', 'atmospherepressure62', 'newtemp102', 'newrel_humidity6', 'atmospherepressure68', 'newwind_dir29', 'newrel_humidity120', 'newtemp20', 'newtemp87', 'var_precip', 'atmospherepressure103', 'newtemp119', 'newwind_dir102', 'var_atmos_press', 'newtemp78', 'newtemp89', 'newrel_humidity41', 'newrel_humidity48', 'windspeed115', 'newtemp95', 'newtemp112', 'newwind_dir53', 'atmospherepressure50', 'windspeed108', 'newrel_humidity50', 'newrel_humidity102', 'std_wind_spd', 'atmospherepressure39', 'newtemp30', 'newwind_dir78', 'newtemp90', 'windspeed107', 'atmospherepressure63', 'newtemp32', 'newtemp75', 'windspeed31', 'newtemp48', 'windspeed4', 'newtemp1', 'windspeed80', 'newwind_dir52', 'atmospherepressure109', 'newtemp97', 'newtemp85', 'windspeed49', 'newwind_dir32', 'newtemp63', 'windspeed56', 'windspeed5', 'newtemp76', 'newrel_humidity103', 'atmospherepressure1', 'newtemp70', 'newrel_humidity17', 'newtemp96', 'atmospherepressure25', 'windspeed42', 'newrel_humidity26', 'atmospherepressure120', 'newrel_humidity61', 'newtemp77', 'newtemp46', 'windspeed97', 'newrel_humidity98', 'newtemp72', 'newrel_humidity33', 'newrel_humidity115', 'newtemp15', 'windspeed32', 'atmospherepressure10']

In [9]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [10]:
lab.fit(stack_all["location"])
stack_all.location  = lab.transform(stack_all.location)
test.location  = lab.transform(test.location)

In [11]:
X=stack_all.drop(["ID",'target'],axis =1)
y=stack_all.target

In [12]:
X = X.filter(cat_vars)
test = test.filter(cat_vars)
errcb2=[]
y_pred_totcb2=[]

fold=KFold(n_splits=20)#15#5#10
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators=7000, eval_metric='RMSE',learning_rate=0.1, 
                            random_seed= 1337, use_best_model=True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test)
    y_pred_totcb2.append(p2)

0:	learn: 39.5567361	test: 39.5567361	test1: 41.0433771	best: 41.0433771 (0)	total: 79.2ms	remaining: 9m 14s
100:	learn: 23.5469830	test: 23.5469830	test1: 28.0323450	best: 28.0323450 (100)	total: 2.87s	remaining: 3m 15s
200:	learn: 20.4105231	test: 20.4105231	test1: 26.2338883	best: 26.2338883 (200)	total: 5.69s	remaining: 3m 12s
300:	learn: 18.4848003	test: 18.4848003	test1: 25.4462557	best: 25.4462557 (300)	total: 8.38s	remaining: 3m 6s
400:	learn: 17.0080398	test: 17.0080398	test1: 24.9109976	best: 24.9109976 (400)	total: 11.2s	remaining: 3m 3s
500:	learn: 15.7736271	test: 15.7736271	test1: 24.4874246	best: 24.4850437 (498)	total: 13.6s	remaining: 2m 56s
600:	learn: 14.7429397	test: 14.7429397	test1: 24.1878810	best: 24.1875425 (596)	total: 15.9s	remaining: 2m 49s
700:	learn: 13.8287594	test: 13.8287594	test1: 23.9214368	best: 23.9214368 (700)	total: 18.3s	remaining: 2m 44s
800:	learn: 13.0575052	test: 13.0575052	test1: 23.6716187	best: 23.6677846 (797)	total: 20.7s	remaining: 2m 4

In [16]:
from datetime import datetime
exp_time = str(datetime.now().replace(second=0, microsecond=0)).replace(" ", "_")
exp_name = "catboost-pseudo"
csv_file_name = "{}_{}".format(exp_name, exp_time)
test_id  = test['ID']
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)

In [17]:
sub = sub[['ID', 'target']]
SUB = "/root/air/sub/"
sub.to_csv(SUB+csv_file_name+'.csv', index=False)